# ClickGraph Graph-Notebook Compatibility Test

This notebook tests ClickGraph's compatibility with AWS Graph Notebook via Bolt protocol.

In [10]:
# Load graph-notebook extension FIRST
%load_ext graph_notebook.magics

The graph_notebook.magics extension is already loaded. To reload it, use:
  %reload_ext graph_notebook.magics


In [9]:
# Install graph-notebook if needed
!pip install graph-notebook


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Test Bolt connection without auth first
from neo4j import GraphDatabase

def test_bolt_no_auth():
    try:
        print("Connecting to ClickGraph via Bolt without auth...")
        # Try without auth first
        driver = GraphDatabase.driver("bolt://localhost:7687")

        with driver.session() as session:
            print("Running test query...")
            result = session.run("MATCH (u:User) RETURN count(u) as user_count")
            record = result.single()
            user_count = record["user_count"]
            print(f"✅ SUCCESS! Bolt connection working. User count: {user_count}")

            # Test another query
            result2 = session.run("MATCH (u:User) RETURN u.name LIMIT 3")
            print("Sample users:")
            for record in result2:
                print(f"  - {record['u.name']}")

        driver.close()
        return True
    except Exception as e:
        print(f"❌ FAILED: {e}")
        return False

def test_bolt_with_auth():
    try:
        print("Connecting to ClickGraph via Bolt with auth...")
        # Try with basic auth - any username/password should work
        driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "test"))

        with driver.session() as session:
            print("Running test query...")
            result = session.run("MATCH (u:User) RETURN count(u) as user_count")
            record = result.single()
            user_count = record["user_count"]
            print(f"✅ SUCCESS! Bolt connection working. User count: {user_count}")

            # Test another query
            result2 = session.run("MATCH (u:User) RETURN u.name LIMIT 3")
            print("Sample users:")
            for record in result2:
                print(f"  - {record['u.name']}")

        driver.close()
        return True
    except Exception as e:
        print(f"❌ FAILED: {e}")
        return False

# Test without auth first
print("=== Testing Bolt without auth ===")
result1 = test_bolt_no_auth()
print(f"Result: {result1}")

print("\n=== Testing Bolt with auth ===")
result2 = test_bolt_with_auth()
print(f"Result: {result2}")

=== Testing Bolt without auth ===
Connecting to ClickGraph via Bolt without auth...
Running test query...
✅ SUCCESS! Bolt connection working. User count: 1033
Sample users:
  - Alice Smith
  - Alice Smith
  - Alice Smith
Result: True

=== Testing Bolt with auth ===
Connecting to ClickGraph via Bolt with auth...
Running test query...
✅ SUCCESS! Bolt connection working. User count: 1033
Sample users:
  - Alice Smith
  - Alice Smith
  - Alice Smith
Result: True


In [ ]:
# Configure graph-notebook to connect to ClickGraph (this loads the extension automatically)
%%graph_notebook_config
{
  "host": "localhost",
  "port": 7687,
  "ssl": false,
  "neo4j": {
    "username": "test",
    "password": "test",
    "auth": true,
    "database": ""
  }
}

UsageError: Line magic function `%%graph_notebook_config` not found.


In [ ]:
# Test direct Bolt connection with neo4j driver
from neo4j import GraphDatabase

def test_connection():
    try:
        # Connect to ClickGraph via Bolt
        driver = GraphDatabase.driver("bolt://localhost:7687", auth=None)
        with driver.session() as session:
            result = session.run("MATCH (u:User) RETURN count(u) as user_count")
            record = result.single()
            print(f"Connection successful! User count: {record['user_count']}")
        driver.close()
        return True
    except Exception as e:
        print(f"Connection failed: {e}")
        return False

test_connection()

In [ ]:
# First test HTTP API to verify ClickGraph is working
import requests

def test_http_api():
    try:
        # Use labeled query since ClickGraph requires labels
        response = requests.post(
            "http://localhost:8080/query",
            json={"query": "MATCH (u:User) RETURN count(u) as user_count"}
        )
        if response.status_code == 200:
            result = response.json()
            print(f"HTTP API working! User count: {result['results'][0]['user_count']}")
            return True
        else:
            print(f"HTTP API error: {response.status_code} - {response.text}")
            return False
    except Exception as e:
        print(f"HTTP connection failed: {e}")
        return False

test_http_api()

HTTP connection failed: 'data'


False

In [ ]:
# Test basic Cypher query
%%opencypher bolt
MATCH (u:User) RETURN count(u) as user_count

In [ ]:
# Test schema discovery
%%opencypher bolt
CALL db.labels() YIELD label RETURN label LIMIT 10

In [ ]:
# Test relationship types
%%opencypher bolt
MATCH (u1:User)-[r]->(u2:User) RETURN type(r) as rel_type, count(r) as rel_count LIMIT 5

# ClickGraph Graph-Notebook Compatibility Test

This notebook tests ClickGraph's compatibility with AWS Graph Notebook via Bolt protocol.

In [ ]:
# Configure graph-notebook to connect to ClickGraph
%%graph_notebook_config
{
  "host": "localhost",
  "port": 7687,
  "ssl": false,
  "neo4j": {
    "username": "neo4j",
    "password": "password",
    "auth": true,
    "database": ""
  }
}

In [ ]:
# Test basic Cypher query
%%opencypher bolt
MATCH (n) RETURN count(n) as node_count

In [ ]:
# Test schema discovery
%%opencypher bolt
CALL db.labels() YIELD label RETURN label LIMIT 10

In [ ]:
# Test relationship types
%%opencypher bolt
CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType LIMIT 10

# Advanced ClickGraph Features Testing

Now let's test more advanced ClickGraph capabilities including graph algorithms, complex queries, and visualization.

In [ ]:
# Test PageRank algorithm (ClickGraph exclusive feature)
%%opencypher bolt
CALL pagerank(
  'MATCH (u:User)-[:FOLLOWS]->(v:User) RETURN id(u) as source, id(v) as target',
  'MATCH (u:User) RETURN id(u) as node',
  0.85,
  10
) YIELD node, rank
RETURN node, rank
ORDER BY rank DESC
LIMIT 10

In [ ]:
# Test shortest path algorithm
%%opencypher bolt
MATCH path = shortestPath((a:User)-[*]-(b:User))
WHERE a.user_id = 1 AND b.user_id = 10
RETURN path, length(path) as path_length

In [ ]:
# Test OPTIONAL MATCH (LEFT JOIN semantics)
%%opencypher bolt
MATCH (u:User)
WHERE u.user_id IN [1, 2, 3]
OPTIONAL MATCH (u)-[:FOLLOWS]->(f:User)
RETURN u.name, u.user_id, f.name as followed_name, f.user_id as followed_id
ORDER BY u.user_id

In [ ]:
# Test complex aggregation with UNION
%%opencypher bolt
MATCH (u:User)
RETURN "Users" as type, count(u) as count, "total_users" as metric
UNION ALL
MATCH ()-[r:FOLLOWS]->()
RETURN "Follows" as type, count(r) as count, "total_follows" as metric
UNION ALL
MATCH (p:Post)
RETURN "Posts" as type, count(p) as count, "total_posts" as metric

In [ ]:
# Test graph visualization - get a small subgraph
%%opencypher bolt
MATCH (u1:User)-[r:FOLLOWS]->(u2:User)
WHERE u1.user_id < 5 AND u2.user_id < 10
RETURN u1, r, u2
LIMIT 20

In [ ]:
# Test variable-length paths (ClickGraph specialty)
%%opencypher bolt
MATCH (u:User)-[*2]->(connected:User)
WHERE u.user_id = 1
RETURN u.name as start_user, connected.name as connected_user, connected.user_id
LIMIT 10

In [ ]:
# Test property-based filtering (ClickGraph optimization)
%%opencypher bolt
MATCH (u:User)
WHERE u.country = 'USA' AND u.is_active = true
RETURN u.name, u.city, u.registration_date
ORDER BY u.registration_date DESC
LIMIT 10

In [ ]:
# Performance test - compare different query patterns
import time
from neo4j import GraphDatabase

def benchmark_query(query, description):
    try:
        driver = GraphDatabase.driver("bolt://localhost:7687")
        start_time = time.time()

        with driver.session() as session:
            result = session.run(query)
            count = len(result.data())

        end_time = time.time()
        elapsed = end_time - start_time

        print(f"{description}: {elapsed:.3f}s, {count} results")
        driver.close()
        return elapsed, count
    except Exception as e:
        print(f"{description}: ERROR - {e}")
        return None, 0

# Test different query patterns
print("=== ClickGraph Performance Benchmark ===")
benchmark_query("MATCH (u:User) RETURN count(u)", "Simple count")
benchmark_query("MATCH (u:User) WHERE u.is_active = true RETURN count(u)", "Filtered count")
benchmark_query("MATCH (u:User)-[:FOLLOWS]->(f:User) RETURN count(u)", "Relationship count")
benchmark_query("MATCH (u:User)-[*2]->(f:User) WHERE u.user_id = 1 RETURN count(f)", "Variable-length path")

# Graph Visualization with graph-notebook

For graph visualization in graph-notebook, you can use these magic commands:

## Available Visualization Commands:
- `%%opencypher bolt` - Execute Cypher queries and display results
- `%%graph_notebook_vis` - Create network visualizations from query results
- `%%neptune_ml` - Machine learning features (if available)

## Visualization Examples:

### 1. Network Graph Visualization:
```python
%%opencypher bolt
MATCH (u1:User)-[r:FOLLOWS]->(u2:User)
WHERE u1.user_id < 10 AND u2.user_id < 10
RETURN u1, r, u2
```

### 2. Force-directed Layout:
```python
%%graph_notebook_vis
MATCH (u1:User)-[r:FOLLOWS]->(u2:User)
WHERE u1.user_id < 5
RETURN u1, r, u2
```

### 3. Path Visualization:
```python
%%opencypher bolt
MATCH path = shortestPath((a:User)-[*]-(b:User))
WHERE a.user_id = 1 AND b.user_id = 5
RETURN path
```

## ClickGraph Features Tested:
✅ **Basic Queries**: MATCH, RETURN, WHERE, LIMIT, ORDER BY
✅ **Relationships**: Pattern matching with relationship types
✅ **Aggregation**: COUNT, SUM, GROUP BY operations
✅ **OPTIONAL MATCH**: LEFT JOIN semantics
✅ **UNION/UNION ALL**: Combining multiple query results
✅ **Variable-Length Paths**: [*n] patterns with optimizations
✅ **Property Filtering**: Schema-based query optimization
✅ **Graph Algorithms**: PageRank, shortestPath
✅ **Performance**: Fast analytical queries on ClickHouse

## Next Steps:
1. Try the visualization cells above
2. Experiment with different query patterns
3. Test with your own data schemas
4. Compare performance with other graph databases